<a href="https://colab.research.google.com/github/luismiguelcasadodiaz/IBM_SkillsBuild_IA_325/blob/main/IA_325_py_cod_ex_24_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendador de Personajes
🎮 "Recomendador de Personajes: ¿Qué tipo de personaje deberías elegir?"

##📘 Enunciado

En este ejercicio trabajarás como desarrollador de sistemas inteligentes para un nuevo videojuego tipo RPG online. El juego permite a los jugadores crear personajes y elegir entre distintos **roles o clases** (por ejemplo: guerrero, mago, arquero, curandero…).

Tu tarea es construir un **modelo de recomendación** que, dado un perfil de jugador (nivel, estilo de combate, número de partidas jugadas, etc.), recomiende qué **tipo de personaje** debería usar, basándose en datos históricos de otros jugadores similares.

## 🧩 Requerimientos

+ Crea una **clase Player**:represente a un jugador con los siguientes
  + Atributos:

    + name: nombre del jugador.
    + level: nivel del jugador (1 a 100).
    + aggressiveness: valor entre 0 y 1 que representa su estilo ofensivo.
    + cooperation: valor entre 0 y 1 que representa cuánto coopera con el equipo.
    + exploration: valor entre 0 y 1 que representa cuánto le gusta explorar el mapa.
    + preferred_class: clase de personaje que suele elegir (solo en los datos de entrenamiento).
  + Métodos
    + .to_features() en la clase para convertir al jugador en una **lista** de características numéricas (sin la clase preferida).

+ Crea una **clase PlayerDataset**:

  + Atributos:
    + una lista de jugadores
  + Métodos:
    + get_X() → lista de listas de características.
    + get_y() → lista de clases preferidas.

+ Crea una clase **ClassRecommender** que use KNN para:
  + Métoodos
    + train : para Entrenar el modelo a partir de un PlayerDataset.
    + predict: para Predecir la mejor clase para un nuevo jugador (predict(player)).
    + get_nearest_neighbors(player) : para obtener los k jugadores más parecidos (get_nearest_neighbors(player)).
    (opcional)
    + best_k : para probar diferentes valores de k
    + evaluate ; para evaluar la precisión del modelo con cross_val_score.



## 🧪 Ejemplo de uso

### Datos de entrenamiento
```python
players = [
    Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
    Player("Bob", 45, 0.4, 0.8, 0.2, "Healer"),
    Player("Cleo", 33, 0.6, 0.4, 0.6, "Archer"),
    Player("Dan", 60, 0.3, 0.9, 0.3, "Healer"),
    Player("Eli", 50, 0.7, 0.2, 0.9, "Mage"),
    Player("Fay", 25, 0.9, 0.1, 0.2, "Warrior"),
]
```

### Nuevo jugador
```python
new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)
```
### Entrenamiento y predicción
```python
dataset = PlayerDataset(players)
recommender = ClassRecommender(n_neighbors=3)
recommender.train(dataset)
```
### Resultado
```python
recommended_class = recommender.predict(new_player)
neighbors_indices = recommender.get_nearest_neighbors(new_player)

print(f"Clase recomendada para {new_player.name}: {recommended_class}")
print("Jugadores similares:")
for i in neighbors_indices:
    print(f"- {players[i].name} ({players[i].preferred_class})")

```

### 🧪 Salida esperada

Clase recomendada para TestPlayer: Archer
Jugadores similares:
- Bob (Healer)
- Cleo (Archer)
- Eli (Mage)

## Importación de librerias

In [219]:
import pandas as pd
import numpy as np
import unittest
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, cross_val_score

## Definición de la clase Player

In [220]:
class Player:
  """
  Representa a un jugador con atributos y método para obtener características.

  Atributos:
    name (str): Nombre del jugador.
    level (int): Nivel del jugador (1 a 100).
    aggressiveness (float): Estilo ofensivo del jugador (0 a 1).
    cooperation (float): Cuánto coopera con el equipo (0 a 1).
    exploration (float): Cuánto le gusta explorar el mapa (0 a 1).
    preferred_class (str): Clase de personaje que suele elegir
      ("Warrior", "Healer", "Archer", "Mage").
  """
  def __init__(self, name, level, aggressiveness, cooperation, exploration, \
               preferred_class=None):
    if not isinstance(name, str) or len(name) == 0:
      raise ValueError("El nombre debe ser una cadena de texto")
    self.name = name
    if not isinstance(level, int) or level < 1 or level > 100:
      raise ValueError("El nivel debe ser entre 1 y 100")
    self.level = level
    if not isinstance(aggressiveness, float) or not (0 <= aggressiveness <= 1):
      raise ValueError("La agressividad debe estar entre 0 y 1")
    self.aggressiveness = aggressiveness
    if not isinstance(cooperation, float) or not (0 <= cooperation <= 1):
      raise ValueError("La cooperacion debe estar entre 0 y 1")
    self.cooperation = cooperation
    if not isinstance(exploration, float) or not (0 <= exploration <= 1):
      raise ValueError("La exploracion debe estar entre 0 y 1")
    self.exploration = exploration
    if preferred_class is not None and preferred_class not in ["Warrior", \
                                                               "Healer",  \
                                                               "Archer",  \
                                                               "Mage"]:
      raise ValueError("La clase debe ser Warrior, Healer, Archer o Mage")
    self.preferred_class = preferred_class
  def to_features(self):
    return [self.level, self.aggressiveness, self.cooperation, self.exploration]



#### Tests para la clase Player

In [221]:
class TestPlayer(unittest.TestCase):

    def test_player_creation_valid(self):
        # Test creating a player with valid attributes
        try:
            player = Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior")
            self.assertEqual(player.name, "Alice")
            self.assertEqual(player.level, 20)
            self.assertEqual(player.aggressiveness, 0.8)
            self.assertEqual(player.cooperation, 0.2)
            self.assertEqual(player.exploration, 0.1)
            self.assertEqual(player.preferred_class, "Warrior")
        except ValueError as e:
            self.fail(f"Valid player creation failed with error: {e}")

    def test_player_creation_invalid_level(self):
        # Test creating a player with an invalid level
        with self.assertRaises(ValueError):
            Player("Bob", 101, 0.4, 0.8, 0.2, "Healer")
        with self.assertRaises(ValueError):
            Player("Bob", 0, 0.4, 0.8, 0.2, "Healer")

    def test_player_creation_invalid_aggressiveness(self):
        # Test creating a player with invalid aggressiveness
        with self.assertRaises(ValueError):
            Player("Cleo", 33, 1.1, 0.4, 0.6, "Archer")
        with self.assertRaises(ValueError):
            Player("Cleo", 33, -0.1, 0.4, 0.6, "Archer")

    def test_player_creation_invalid_cooperation(self):
        # Test creating a player with invalid cooperation
        with self.assertRaises(ValueError):
            Player("Dan", 60, 0.3, 1.2, 0.3, "Healer")
        with self.assertRaises(ValueError):
            Player("Dan", 60, 0.3, -0.2, 0.3, "Healer")

    def test_player_creation_invalid_exploration(self):
        # Test creating a player with invalid exploration
        with self.assertRaises(ValueError):
            Player("Eli", 50, 0.7, 0.2, 1.3, "Mage")
        with self.assertRaises(ValueError):
            Player("Eli", 50, 0.7, 0.2, -0.3, "Mage")

    def test_player_creation_invalid_preferred_class(self):
        # Test creating a player with an invalid preferred class
        with self.assertRaises(ValueError):
            Player("Fay", 25, 0.9, 0.1, 0.2, "InvalidClass")

    def test_to_features(self):
        # Test the to_features method
        player = Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior")
        features = player.to_features()
        self.assertEqual(features, [20, 0.8, 0.2, 0.1])


In [222]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..F.............
FAIL: test_predict (__main__.TestClassRecommender.test_predict)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-214-9d74e682c354>", line 27, in test_predict
    self.assertIsInstance(recommended_class, (list, np.ndarray))
AssertionError: np.str_('Archer') is not an instance of (<class 'list'>, <class 'numpy.ndarray'>)

----------------------------------------------------------------------
Ran 16 tests in 0.028s

FAILED (failures=1)


## Definición de la clase PlayerDataset

In [223]:
class PlayerDataset:
  """Representa un conjunto de datos de jugadores.

  Atributos:
    players
  """
  def __init__(self, players):
    if not isinstance(players, list):
      raise ValueError("Los jugadores deben ser una lista")
    self.players = players

  def get_X(self):
    return [player.to_features() for player in self.players]

  def get_y(self):
    return [player.preferred_class for player in self.players]

#### Test para la clase PlayerDataset

In [224]:
class TestPlayerDataset(unittest.TestCase):

    def test_dataset_creation_valid(self):
        # Test creating a PlayerDataset with valid input
        players = [
            Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
            Player("Bob", 45, 0.4, 0.8, 0.2, "Healer")
        ]
        try:
            dataset = PlayerDataset(players)
            self.assertEqual(len(dataset.players), 2)
            self.assertEqual(dataset.players[0].name, "Alice")
        except ValueError as e:
            self.fail(f"Valid PlayerDataset creation failed with error: {e}")

    def test_dataset_creation_invalid(self):
        # Test creating a PlayerDataset with invalid input
        with self.assertRaises(ValueError):
            PlayerDataset("not a list")

    def test_get_X(self):
        # Test the get_X method
        players = [
            Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
            Player("Bob", 45, 0.4, 0.8, 0.2, "Healer")
        ]
        dataset = PlayerDataset(players)
        X = dataset.get_X()
        self.assertEqual(X, [[20, 0.8, 0.2, 0.1], [45, 0.4, 0.8, 0.2]])

    def test_get_y(self):
        # Test the get_y method
        players = [
            Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
            Player("Bob", 45, 0.4, 0.8, 0.2, "Healer")
        ]
        dataset = PlayerDataset(players)
        y = dataset.get_y()
        self.assertEqual(y, ["Warrior", "Healer"])

In [225]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..F.............
FAIL: test_predict (__main__.TestClassRecommender.test_predict)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-214-9d74e682c354>", line 27, in test_predict
    self.assertIsInstance(recommended_class, (list, np.ndarray))
AssertionError: np.str_('Archer') is not an instance of (<class 'list'>, <class 'numpy.ndarray'>)

----------------------------------------------------------------------
Ran 16 tests in 0.024s

FAILED (failures=1)


## Definición de la clase ClassRecommender

In [226]:
class ClassRecommender:
  """Recomendador de clases de personaje basado en jugadores similares usando KNN.

  Atributos:
    n_neighbors (int): Número de vecinos a considerar para la recomendación.
    knn (KNeighborsClassifier): El modelo KNN entrenado.
  """
  def __init__(self, n_neighbors=5):
    self.n_neighbors = n_neighbors
    self.knn = None

  def train(self, dataset):
    """Entrena el modelo KNN con los datos del dataset.

    Args:
      dataset (PlayerDataset): El dataset de jugadores a usar para entrenar.
    """
    X = np.array(dataset.get_X())
    y = np.array(dataset.get_y())
    self.knn = KNeighborsClassifier(n_neighbors=self.n_neighbors)
    self.knn.fit(X, y)

  def predict(self, new_player):
    """Predice la clase de personaje recomendada para un nuevo jugador.

    Args:
      new_player (Player): El nuevo jugador para el que hacer la predicción.

    Returns:
      str: La clase de personaje recomendada.

    Raises:
      ValueError: Si el modelo no ha sido entrenado.
    """
    if self.knn is None:
      raise ValueError("El modelo no ha sido entrenado")
    new_player_features = np.array(new_player.to_features()).reshape(1, -1)
    return self.knn.predict(new_player_features)[0]

  def get_nearest_neighbors(self, new_player):
    """Obtiene los índices de los vecinos más cercanos a un nuevo jugador.

    Args:
      new_player (Player): El nuevo jugador.

    Returns:
      np.ndarray: Un array con los índices de los k vecinos más cercanos.

    Raises:
      ValueError: Si el modelo no ha sido entrenado.
    """
    if self.knn is None:
      raise ValueError("El modelo no ha sido entrenado")
    new_player_features = np.array(new_player.to_features()).reshape(1, -1)
    _, indices = self.knn.kneighbors(new_player_features)
    return indices.flatten()

  def best_k(self, dataset):
    """Encuentra el mejor valor de k para el modelo usando validación cruzada.

    Args:
      dataset (PlayerDataset): El dataset de jugadores para evaluar.

    Returns:
      int: El valor de k que resultó en el menor error.
    """
    X = np.array(dataset.get_X())
    y = np.array(dataset.get_y())
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    tasa_errores = []
    for k in range(1, X_train.shape[0]):
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(X_train, y_train)
      y_pred = knn.predict(X_test)
      tasa_errores.append(np.mean(y_pred != y_test))
    print(tasa_errores) # Considerar si esto es necesario en un método
    min_error = min(tasa_errores)
    best_k = tasa_errores.index(min_error) + 1
    return best_k

  def evaluate(self, dataset):
    """Evalúa la precisión del modelo usando validación cruzada.

    Args:
      dataset (PlayerDataset): El dataset de jugadores para evaluar.

    Returns:
      np.ndarray: Un array con las puntuaciones de precisión de cada fold.

    Raises:
      ValueError: Si el modelo no ha sido entrenado.
    """
    if self.knn is None:
        raise ValueError("El modelo no ha sido entrenado")
    X = np.array(dataset.get_X())
    y = np.array(dataset.get_y())
    scores = cross_val_score(self.knn, X, y, cv=5)
    return scores


#### Tests para la clase ClassRecommender

In [227]:
class TestClassRecommender(unittest.TestCase):

    def setUp(self):
        # Set up a sample dataset and recommender for testing
        self.players = [
            Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
            Player("Bob", 45, 0.4, 0.8, 0.2, "Healer"),
            Player("Cleo", 33, 0.6, 0.4, 0.6, "Archer"),
            Player("Dan", 60, 0.3, 0.9, 0.3, "Healer"),
            Player("Eli", 50, 0.7, 0.2, 0.9, "Mage"),
            Player("Fay", 25, 0.9, 0.1, 0.2, "Warrior"),
        ]
        self.dataset = PlayerDataset(self.players)
        self.recommender = ClassRecommender(n_neighbors=3)
        self.recommender.train(self.dataset)

    def test_train(self):
        # Test if the model is trained (knn attribute is not None)
        self.assertIsNotNone(self.recommender.knn)

    def test_predict(self):
        # Test the predict method with a new player
        new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)
        recommended_class = self.recommender.predict(new_player)
        # Since KNN depends on the data and k, we can't assert a specific class
        # but we can check if the output is a list or array (from the model)
        self.assertIsInstance(recommended_class, (list, np.ndarray))
        # A more specific test would require knowing the expected output for this
        # specific new_player and dataset. For example:
        # self.assertEqual(recommended_class[0], "Archer") # If "Archer" is the expected class

    def test_predict_before_training(self):
        # Test if predict raises an error before training
        recommender_untrained = ClassRecommender(n_neighbors=3)
        new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)
        with self.assertRaises(ValueError):
            recommender_untrained.predict(new_player)

    def test_get_nearest_neighbors(self):
        # Test the get_nearest_neighbors method
        new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)
        neighbors_indices = self.recommender.get_nearest_neighbors(new_player)
        # We can check if the number of neighbors matches n_neighbors
        self.assertEqual(len(neighbors_indices), self.recommender.n_neighbors)
        # We can also check if the indices are within the bounds of the dataset
        for index in neighbors_indices:
            self.assertGreaterEqual(index, 0)
            self.assertLess(index, len(self.players))

    def test_get_nearest_neighbors_before_training(self):
        # Test if get_nearest_neighbors raises an error before training
        recommender_untrained = ClassRecommender(n_neighbors=3)
        new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)
        with self.assertRaises(ValueError):
            recommender_untrained.get_nearest_neighbors(new_player)

    # Note: Testing best_k and evaluate is more complex as it involves
    # splitting data and cross-validation, which can be non-deterministic
    # depending on the random state. You would typically mock or carefully
    # set up the data to test these methods reliably. For a basic test,
    # you could check if they return values of the expected type.

    # def test_best_k(self):
    #     # Basic test for best_k - check if it returns an integer
    #     best_k_value = self.recommender.best_k(self.dataset)
    #     self.assertIsInstance(best_k_value, int)
    #     self.assertGreaterEqual(best_k_value, 1) # Assuming k is at least 1

    # def test_evaluate(self):
    #     # Basic test for evaluate - check if it returns a numpy array
    #     scores = self.recommender.evaluate(self.dataset)
    #     self.assertIsInstance(scores, np.ndarray)
    #     self.assertEqual(len(scores), 5) # Assuming cv=5

# To run the tests, uncomment the following line after the previous unittest.main calls:


In [228]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..F.............
FAIL: test_predict (__main__.TestClassRecommender.test_predict)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-227-9d74e682c354>", line 27, in test_predict
    self.assertIsInstance(recommended_class, (list, np.ndarray))
AssertionError: np.str_('Archer') is not an instance of (<class 'list'>, <class 'numpy.ndarray'>)

----------------------------------------------------------------------
Ran 16 tests in 0.033s

FAILED (failures=1)


## Datos de entrenamiento y uso

In [229]:
players = [
    Player("Alice", 20, 0.8, 0.2, 0.1, "Warrior"),
    Player("Bob", 45, 0.4, 0.8, 0.2, "Healer"),
    Player("Cleo", 33, 0.6, 0.4, 0.6, "Archer"),
    Player("Dan", 60, 0.3, 0.9, 0.3, "Healer"),
    Player("Eli", 50, 0.7, 0.2, 0.9, "Mage"),
    Player("Fay", 25, 0.9, 0.1, 0.2, "Warrior"),
]
new_player = Player("TestPlayer", 40, 0.6, 0.3, 0.8)

## Ejemplo de uso

In [230]:
dataset = PlayerDataset(players)
recommender = ClassRecommender(n_neighbors=3)
recommender.train(dataset)
recommended_class = recommender.predict(new_player)
neighbors_indices = recommender.get_nearest_neighbors(new_player)
print("EL mejor K para este dataset es ",recommender.best_k(dataset))

print(f"Clase recomendada para {new_player.name}: {recommended_class}")
print("Jugadores similares:")
for i in neighbors_indices:
    print(f"- {players[i].name} ({players[i].preferred_class})")


[np.float64(0.5), np.float64(1.0), np.float64(1.0)]
EL mejor K para este dataset es  1
Clase recomendada para TestPlayer: Archer
Jugadores similares:
- Bob (Healer)
- Cleo (Archer)
- Eli (Mage)
